In [1]:
gpu_info_examine = !nvidia-smi
gpu_info_examine = '\n'.join(gpu_info_examine)
if gpu_info_examine.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info_examine)

Tue Jul 13 15:26:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    44W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install tqdm
!pip install numpy

In [4]:
from transformers import AutoTokenizer, AutoModel, BartForConditionalGeneration, BartTokenizerFast
from datasets import load_dataset, load_metric
import torch as torch
from tqdm import tqdm
from datasets import load_metric
import numpy as np

In [5]:
model_checkpoint = 'a1noack/bart-large-gigaword' #'facebook/bart-base'  #'facebook/bart-large-cnn'
tokenizer = BartTokenizerFast.from_pretrained(model_checkpoint)
model = BartForConditionalGeneration.from_pretrained(model_checkpoint, return_dict=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)


cuda


In [6]:
# test = load_dataset("wikihow", "all", data_dir="/scratch/nm3571", split='test')
# test = load_dataset("gigaword", split='test')
# test = load_dataset("big_patent", "g", split='test')
test = load_dataset("xsum", split='test')
# test = load_dataset("cnn_daily",cache_dir='/scratch/yk2516/cache/', split='test[:]')
# test = load_dataset("cnn_dailymail", name='3.0.0',split='test', cache_dir='/scratch/yk2516/cache/')

Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)


In [7]:
print(test)
print(model.config.max_length)

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 11334
})
32


In [8]:
encodings =  tokenizer(test['document'], return_tensors='pt', padding=True, truncation=True, max_length=1024).to(device)

In [9]:
model = model.to(device)
model.eval()
number_beams = 8
# print(encodings['input_ids'])
# print(encodings['input_ids'].cpu().detach().numpy())
# print(torch.from_numpy(encodings['input_ids'].cpu().detach().numpy()).to(device))
# print(type(encodings['input_ids'][0].item()))
# print("Wikihow vocab size: ", result.scores[0].shape[1])
print("Input ids size", encodings['input_ids'].shape)
ids = encodings['input_ids'].cpu().detach().numpy()
attention_ids = encodings['attention_mask'].cpu().detach().numpy()
log_sent = []
num_words = []
print("number of samples:", ids.shape[0])

Input ids size torch.Size([11334, 1024])
number of samples: 11334


In [10]:
!pip install rouge_score

In [11]:
rouge = load_metric('rouge')

In [12]:
# for i in range(ids.shape[0]):
#     print(i)
#     input_id = np.array([ids[i]])
#     input_id = torch.from_numpy(input_id).to(device)
#     attention_id = np.array([attention_ids[i]])
#     attention_id = torch.from_numpy(attention_id).to(device)
#     with torch.no_grad():
#         result = model.generate(input_ids=input_id, attention_mask=attention_id, num_beams=number_beams, return_dict_in_generate=True, max_length=model.config.max_length, output_scores=True, output_attentions=True)
#     print(len(result))
#     break

0


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


6


In [86]:
# gen_output = tokenizer.batch_decode(result[0], skip_special_tokens=True)# , clean_up_tokenization_spaces=False)

In [87]:
# rouge.add(prediction=gen_output[0], reference=test['summary'][0])
# # rouge.add(prediction=['i see'], reference=['i see'])
# # metric_r.add_batch(predictions=['alright man'], references=['sure'])


In [12]:
ids.shape[0]

11334

In [13]:
for i in range(ids.shape[0]):
    print(i)
    input_id = np.array([ids[i]])
    input_id = torch.from_numpy(input_id).to(device)
    attention_id = np.array([attention_ids[i]])
    attention_id = torch.from_numpy(attention_id).to(device)
    with torch.no_grad():
        result = model.generate(input_ids=input_id, attention_mask=attention_id, num_beams=number_beams, return_dict_in_generate=True, max_length=model.config.max_length, output_scores=True, output_attentions=True)
    
    generated_summary = tokenizer.batch_decode(result[0], skip_special_tokens=True) #, clean_up_tokenization_spaces=False)
    
    # rouge.add_batch(predictions=generated_summary, references=test['summary'][i])
    rouge.add(prediction=generated_summary[0], reference=test['summary'][i])


0


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: ignored

In [14]:
score = rouge.compute()

In [15]:
print('Rouge: ')
print(score)

Rouge: 
{'rouge1': AggregateScore(low=Score(precision=0.28755039554786616, recall=0.10398658312866452, fmeasure=0.14971646311880116), mid=Score(precision=0.294468547191987, recall=0.10660070001983891, fmeasure=0.15324686973992804), high=Score(precision=0.30192914297151247, recall=0.1091981823437053, fmeasure=0.15687833054575379)), 'rouge2': AggregateScore(low=Score(precision=0.05625541216793321, recall=0.01942536496001024, fmeasure=0.02821968256499428), mid=Score(precision=0.060261273296686374, recall=0.020832963957024307, fmeasure=0.030255513148322463), high=Score(precision=0.06456153957060345, recall=0.022440516157859688, fmeasure=0.03242304059854147)), 'rougeL': AggregateScore(low=Score(precision=0.24625770290230647, recall=0.08894001587658515, fmeasure=0.127935906870553), mid=Score(precision=0.25248569773864854, recall=0.09133446318621014, fmeasure=0.13118136870718222), high=Score(precision=0.25885243941462305, recall=0.09394555128714051, fmeasure=0.1347302759721665)), 'rougeLsum':